# Colorbot Solutions 

Here are the solutions to the exercises available at the colorbot notebook.

In order to compare the models we encourage you to use Tensorboard and also use play_colorbot.py --model_dir=path_to_your_model to play with the models and check how it does with general words other than color words.


## EXERCISE EXPERIMENT

When using experiments you should make sure you repeat the datasets the number of epochs desired since the experiment will "run the for loop for you". Also, you can add a parameter to run a number of steps instead, it will run until the dataset ends or the number of steps.

You can add this cell to your colorbot notebook and run it.

In [ ]:
# small important detail, to train properly with the experiment you need to
# repeat the dataset the number of epochs desired
train_input_fn = get_input_fn(TRAIN_INPUT, BATCH_SIZE, num_epochs=40)

# create experiment
def generate_experiment_fn(run_config, hparams):
    estimator = tf.estimator.Estimator(model_fn=model_fn, config=run_config)
    return tf.contrib.learn.Experiment(
        estimator,
        train_input_fn=train_input_fn,
        eval_input_fn=test_input_fn
    )

learn_runner.run(generate_experiment_fn, run_config=tf.contrib.learn.RunConfig(model_dir='model_dir'))

## EXERCISE DATASET

0. Run the colorbot experiment and notice the choosen model_dir
1. Below is the input function definition,we don't need some of the auxiliar functions anymore
2. Add this cell and then add the solution to the EXERCISE EXPERIMENT
3. choose a different model_dir and run the cells
4. Copy the model_dir of the two models to the same path
5. tensorboard --logdir=path

In [ ]:
def get_input_fn(csv_file, batch_size, num_epochs=1, shuffle=True):
    def _parse(line):
        # each line: name, red, green, blue
        # split line
        items = tf.string_split([line],',').values

        # get color (r, g, b)
        color = tf.string_to_number(items[1:], out_type=tf.float32) / 255.0

        # split color_name into a sequence of characters
        color_name = tf.string_split([items[0]], '')
        length = color_name.indices[-1, 1] + 1 # length = index of last char + 1
        color_name = color_name.values
        return color, color_name, length

    def input_fn():
        # https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/data
        dataset = (
            tf.contrib.data.TextLineDataset(csv_file) # reading from the HD
            .skip(1) # skip header
            .map(_parse) # parse text to variables
            .padded_batch(batch_size, padded_shapes=([None], [None], []),
                               padding_values=(0.0, chr(0), tf.cast(0, tf.int64)))
            
            .repeat(num_epochs) # repeat dataset the number of epochs
        )
        
        # for our "manual" test we don't want to shuffle the data
        if shuffle:
            dataset = dataset.shuffle(buffer_size=100000)

        # create iterator
        color, color_name, length = dataset.make_one_shot_iterator().get_next()

        features = {
            COLOR_NAME_KEY: color_name,
            SEQUENCE_LENGTH_KEY: length,
        }

        return features, color
    return input_fn

As a result you will see something like:

![](imgs/dataset_exercise_sol.png)

We called the original model "sorted_batch" and the model using the simplified input function as "simple_batch"

Notice that both models have basically the same loss in the last step, but the "sorted_batch" model runs way faster , notice the `global_step/sec` metric, it measures how many steps the model executes per second. Since the "sorted_batch" has a larger `global_step/sec` it means it trains faster. 

If you don't belive me you can change Tensorboard to compare the models in a "relative" way, this will compare the models over time. See result below.

![](imgs/dataset_exercise_relative_sol.png)

## EXERCISE HYPERPARAMETERS

This one is more personal, what you see will depends on what you change in the model.
Below is a very simple example we just changed the model to use a GRUCell, just in case...

In [ ]:
def get_model_fn(rnn_cell_sizes,
                 label_dimension,
                 dnn_layer_sizes=[],
                 optimizer='SGD',
                 learning_rate=0.01):
    
    def model_fn(features, labels, mode):
        
        color_name = features[COLOR_NAME_KEY]
        sequence_length = tf.cast(features[SEQUENCE_LENGTH_KEY], dtype=tf.int32) # int64 -> int32
        
        # ----------- Preparing input --------------------
        # Creating a tf constant to hold the map char -> index
        # this is need to create the sparse tensor and after the one hot encode
        mapping = tf.constant(CHARACTERS, name="mapping")
        table = tf.contrib.lookup.index_table_from_tensor(mapping, dtype=tf.string)
        int_color_name = table.lookup(color_name)
        
        # representing colornames with one hot representation
        color_name_onehot = tf.one_hot(int_color_name, depth=len(CHARACTERS) + 1)
        
        # ---------- RNN -------------------
        # Each RNN layer will consist of a GRU cell
        rnn_layers = [tf.nn.rnn_cell.GRUCell(size) for size in rnn_cell_sizes]
        
        # Construct the layers
        multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
        
        # Runs the RNN model dynamically
        # more about it at: 
        # https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn
        outputs, final_state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
                                                 inputs=color_name_onehot,
                                                 sequence_length=sequence_length,
                                                 dtype=tf.float32)

        # Slice to keep only the last cell of the RNN
        last_activations = rnn_common.select_last_activations(outputs,
                                                              sequence_length)

        # ------------ Dense layers -------------------
        # Construct dense layers on top of the last cell of the RNN
        for units in dnn_layer_sizes:
            last_activations = tf.layers.dense(
              last_activations, units, activation=tf.nn.relu)
        
        # Final dense layer for prediction
        predictions = tf.layers.dense(last_activations, label_dimension)

        # ----------- Loss and Optimizer ----------------
        loss = None
        train_op = None

        if mode != tf.estimator.ModeKeys.PREDICT:    
            loss = tf.losses.mean_squared_error(labels, predictions)
    
        if mode == tf.estimator.ModeKeys.TRAIN:    
            train_op = tf.contrib.layers.optimize_loss(
              loss,
              tf.contrib.framework.get_global_step(),
              optimizer=optimizer,
              learning_rate=learning_rate)
        
        return model_fn_lib.EstimatorSpec(mode,
                                           predictions=predictions,
                                           loss=loss,
                                           train_op=train_op)
    return model_fn

Below is the tensorboard comparison of a model using a GRUCell called "gru" and a model using LSTMCell called "simple_batch".

![](imgs/hyperparameters_exercise_sol.png)
